In [1]:
from ultralytics import YOLO
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score

In [36]:
modelDir = "best.pt"
sourceImgDir = "../../images/test"
#sourceImgDir = "./images"
csvName = "output.csv"

In [21]:
resultsList = []

In [38]:
try:
    images = os.listdir(sourceImgDir)
    images = [os.path.join(sourceImgDir, x) for x in images]
except FileNotFoundError as e:
    print(e)

In [24]:
model = YOLO(modelDir)

In [31]:
for image in images:
    imgDir = os.path.join(sourceImgDir, image)
    results = model.predict(
        source = imgDir,
        imgsz = 1024,
        conf = 0.0001,
        iou = 0.6,
        max_det = 1,
    )
    image_id = [image[:-4]]
    
    for r in results:
        conf = r.boxes.conf.tolist()
        box = r.boxes.xyxy.tolist()
        if box:
            resultsList.append(image_id + box[0] + conf)


image 1/1 c:\Users\engli\Documents\School Stuff\Y2S1\NPSC2001\Mini_VinDr_CXR_Dataset\manualVal\images\fdc79ca123f665375a024d95fce24af4.png: 1024x1024 1 Cardiomegaly, 732.6ms
Speed: 4.5ms preprocess, 732.6ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)


In [39]:
results = model.predict(
        source = images,
        imgsz = 1024,
        conf = 0.0001,
        iou = 0.6,
        max_det = 1,
        device = "cuda:0",
        stream=True
    )

In [40]:
resultsList

[['002a34c58c5b758217ed1f584ccbcfe9',
  377.5561828613281,
  465.7619323730469,
  802.10693359375,
  649.0537109375,
  0.0003124689101241529],
 ['004f33259ee4aef671c2b95d54e4be68',
  430.89300537109375,
  497.1188049316406,
  729.8722534179688,
  603.0970458984375,
  0.0002474078210070729],
 ['008bdde2af2462e86fd373a445d0f4cd',
  487.47662353515625,
  490.8783264160156,
  790.5674438476562,
  642.6561279296875,
  0.5089626312255859],
 ['009bc039326338823ca3aa84381f17f1',
  372.41021728515625,
  442.6533203125,
  750.1553344726562,
  569.6885375976562,
  0.02358906716108322],
 ['00a2145de1886cb9eb88869c85d74080',
  387.20587158203125,
  531.1728515625,
  854.7800903320312,
  709.427490234375,
  0.7204761505126953],
 ['00b7e6bfa4dc1fe9ddd0ce74743e38c2',
  371.6971740722656,
  481.67926025390625,
  706.6717529296875,
  659.6911010742188,
  0.28371864557266235],
 ['002a34c58c5b758217ed1f584ccbcfe9',
  377.5561828613281,
  465.7619323730469,
  802.10693359375,
  649.0537109375,
  0.00031246

In [41]:
df = pd.DataFrame(resultsList, columns = ["image_id", "x_min", "y_min", "x_max", "y_max", "conf"])
df.to_csv(csvName, sep = ",", index = None)

In [50]:
groundTruth = pd.read_csv(r"C:\Users\engli\Documents\School Stuff\Y2S1\NPSC2001\Mini_VinDr_CXR_Dataset\subset\c-subset\mergedTest.csv")

In [51]:
groundTruth = groundTruth[groundTruth["class_name"] == "Cardiomegaly"]

In [58]:
groundTruth.sort_index()

,image_id,class_name,x_min,y_min,x_max,y_max
0,e0dc2e79105ad93532484e956ef8a71a,Cardiomegaly,328.257771,554.673173,819.965088,820.039027
1,42da2c134b53cb5594774d3d29faac59,Cardiomegaly,304.821814,495.098180,840.552539,719.683986
2,c7179539654a1b3b7977e56e7e3009d5,Cardiomegaly,285.719686,509.770654,727.643238,730.732429
3,40497da14aafc428210b623fc8445288,Cardiomegaly,377.327945,636.684680,845.837598,842.275413
4,91123b24b8a4c289820bbb2dc6dabdfe,Cardiomegaly,377.604606,559.853055,836.684550,728.742221
...,...,...,...,...,...,...
304,0792bb20718b8006f78094ae449c3c96,Cardiomegaly,375.441169,545.212195,835.576246,735.444718
305,49381febdd648ccc7a190965fe5180bd,Cardiomegaly,421.306900,404.628208,816.989185,600.620368
306,f6fa36dcae4cfd831f6d5a6bd4520a59,Cardiomegaly,384.559697,565.420752,827.927711,795.549864
307,13b9d029ff4c874e22f539c174ae96ff,Cardiomegaly,355.562133,442.954950,881.399971,725.642209


In [59]:
df.sort_index()

<bound method DataFrame.sort_values of                              image_id       x_min       y_min       x_max  \
0    002a34c58c5b758217ed1f584ccbcfe9  377.556183  465.761932  802.106934   
1    004f33259ee4aef671c2b95d54e4be68  430.893005  497.118805  729.872253   
2    008bdde2af2462e86fd373a445d0f4cd  487.476624  490.878326  790.567444   
3    009bc039326338823ca3aa84381f17f1  372.410217  442.653320  750.155334   
4    00a2145de1886cb9eb88869c85d74080  387.205872  531.172852  854.780090   
..                                ...         ...         ...         ...   
658  4dcaf539b6497ad02f2734e1b78c9678  451.775299  498.447998  861.844360   
659  4de27ac8b1c575b620cc16780030086d  422.901642  643.168701  770.781006   
660  4e36b920e32cc309388138261b94f741  403.243347  558.848206  778.396545   
661  4e6269ee38f7c8afc70792e62d0a05e8  375.455566  480.739624  702.640625   
662  fdc79ca123f665375a024d95fce24af4  393.135040  588.795715  888.332520   

          y_max      conf  
0    649